In [ ]:
# compute count of rides and duration per rider, per month
rider_stats = spark.sql(
    """
    SELECT rider_id, MONTH(started_at) as month, YEAR(started_at) as year, count(*) as num_rides, AVG( DATEDIFF (seconds, started_at, ended_at)) * 60 as minutes_spent
    FROM trips
    GROUP by rider_id, MONTH(started_at), YEAR(started_at);
    """
)
rider_stats.createOrReplaceTempView("rider_stats")

In [ ]:
# average statistics over months
rider_stats_month = spark.sql(
"""
    SELECT rider_id, avg(num_rides) as avg_rides_month,
     avg(minutes_spent) as avg_time_spent_month
    from rider_stats
    GROUP BY rider_id
"""
)
display(rider_stats_month.head())
rider_stats_month.createOrReplaceTempView("rider_stats_month")

In [ ]:
%sql
insert into dim_rider
(
  select riders.rider_id as id,
  first as firstname,
  last as lastname,
  address,
  birthday,
  account_start_date,
  account_end_date,
  is_member,
  avg_rides_month,
  avg_time_spent_month,
  floor (months_between (birthday, account_start_date) / 12)
  as rider_age_account_start
  from riders 
  join rider_stats_month on (riders.rider_id = rider_stats_month.rider_id)
)

In [ ]:
%sql
select * from dim_rider limit 10